In [ ]:
from PlanningPbm import getPlanningPbm
from branchAndBound import TreeNode, get_constraint, get_constraint_from_node, branch_and_bound
from ranking import getScheduleFromRanking, getRanking
from utils import PriorityQueue
import time
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import time
import gurobipy
from gurobipy import *
from gurobipy import quicksum, GRB
import ast

# Get job shop problem and a first schedule
PLAN = getPlanningPbm("mockel2")
SD = PLAN.getFirstSchedule(randomFlag=False) # schedule deter

In [ ]:
def createModelInitial(schedule, constraintsBB = [None],setS=[]):
    # Create the inital model for the dive method.
    # Input:
    #   - schedule: a schedule
    #   - constraintsBB: a list of constraints fromt the constraint strategy
    #                    It is just used to add the first constraint found by the strategy
    #   - setS: a list of 0 and 1, 1 if the constraint is used, 0 otherwise
    # Output:
    #   - model: the model

    # Create a new model
    # ------------------
    model = gurobipy.Model("schedule")
    model.Params.LogToConsole = 0 # disable console output

    # Create variables
    # ------------------
    Completion = {}
    End= {}
    for m in range(PLAN.nbMach):
        for o in range(PLAN.nbOpPerMach[m]):
            Completion[m,o]=model.addVar(vtype=GRB.CONTINUOUS,lb=0,name="c_%s_%s"%(m,o))
    for j in range(PLAN.nbJobs):
        End[j]=model.addVar(vtype=GRB.CONTINUOUS,lb=0,name="e_%s"%(j))

    # Set objective
    # -------------
    model.setObjective(quicksum([(End[j]) for j in range(PLAN.nbJobs)]), GRB.MINIMIZE)²
    
    # Add initial constraints 
    # -----------------------

    # Constraints on completion time individually for each operation
    for m in range(PLAN.nbMach):
        for o in range(0,PLAN.nbOpPerMach[m]):
            model.addConstr(Completion[m,o] >= schedule[m][o][2] + schedule[m][o][3])

    idxOp = PLAN.getOpIndexInSched(schedule)
    for j in range(PLAN.nbJobs):

        # deal with jobs with no operation (i.e. op only on machine with infinite capacity)
        if PLAN.nbOpPerJob[j] == 0:
            continue

        # Get end 
        oIdx = max(PLAN.nbOpPerJob[j]-1, 0)
        m = idxOp[j][oIdx][0]
        mo = idxOp[j][oIdx][1]
        model.addConstr(End[j] == Completion[m,mo]+schedule[m][mo][4])

        # Precedence constraints
        for o in range(1,PLAN.nbOpPerJob[j]):            
            m1, mo1 = idxOp[j][o][0], idxOp[j][o][1] # operation o of job j
            m0, mo0 = idxOp[j][o-1][0], idxOp[j][o-1][1]  # operation o-1 of job j
            model.addConstr(Completion[m1,mo1] >= Completion[m0,mo0] + schedule[m0][mo0][4] + schedule[m1][mo1][2] + schedule[m1][mo1][3])

    # Add constraints depending on the set S
    # --------------------------------------
    if len(setS)>0: # Constraint from testing
        
        if setS[0] == 1: # Contraints on each pair of operation on the same machine            
            for m in range(PLAN.nbMach):
                for o1 in range(0,PLAN.nbOpPerMach[m]):
                    for o2 in range(o1+1,PLAN.nbOpPerMach[m]):
                        lhs = Completion[m,o1] * schedule[m][o1][3] + Completion[m,o2] * schedule[m][o2][3]
                        rhs1 = 1/2 * ((schedule[m][o1][3] + schedule[m][o2][3])**2)
                        rhs2 = 1/2 * (schedule[m][o1][3]**2 + schedule[m][o2][3]**2)
                        model.addConstr(lhs >= rhs1 + rhs2)

        if setS[1] == 1 or setS[2] == 1: 
            for m in range(PLAN.nbMach):
                lhs = 0
                rhs1 = 0
                rhs2 = 0
                for o in range(PLAN.nbOpPerMach[m]):
                    lhs += Completion[m,o] * schedule[m][o][3]
                    rhs1 += schedule[m][o][3]
                    rhs2 += schedule[m][o][3]**2
                if setS[1] == 1: # constraints on all operations
                    model.addConstr(lhs >= 1/2 * rhs1**2 +  1/2 * rhs2)
                
                if setS[2] == 1: # constraint on all operations expect one                    
                    for o in range(PLAN.nbOpPerMach[m]):
                        tmpLhs = lhs - Completion[m,o] * schedule[m][o][3]
                        tmpRhs1 = rhs1 - schedule[m][o][3]
                        tmpRhs2 = rhs2 - schedule[m][o][3]**2
                        model.addConstr(tmpLhs >= 1/2 * tmpRhs1**2 +  1/2 * tmpRhs2)
        
        sizeSet = []
        startNb = 2 
        for i in range(3,len(setS)):
            if setS[i] ==1:
                sizeSet.append(startNb)
            startNb += 1
        
        if len(sizeSet)>0: # constraints on successive set of operations on the same machine

            for m in range(PLAN.nbMach):
                for size in sizeSet:
                    for o1 in range(0,PLAN.nbOpPerMach[m]-size+1):
                        lhs = 0
                        rhs1 = 0
                        rhs2 = 0
                        for o in range(size):
                            lhs += Completion[m,o1+o] * schedule[m][o1+o][3]
                            rhs1 += schedule[m][o1+o][3]
                            rhs2 += schedule[m][o1+o][3]**2
                        model.addConstr(lhs >= 1/2 * rhs1**2 +  1/2 * rhs2)

    # Constraints from b&b
    # --------------------
    if constraintsBB[0] is not None:  # Constraints add from B&B
        nbConst = int(len(constraintsBB)/4)
        for i in range(nbConst):
            
            m = constraintsBB[4*i]
            o1 = constraintsBB[4*i+1]
            o2 = constraintsBB[4*i+2]
            op = constraintsBB[4*i+3]
            if op == 0:
                model.addConstr(Completion[m,o1] >= Completion[m,o2] + schedule[m][o1][3])              
            else:
                model.addConstr(Completion[m,o2] >= Completion[m,o1] + schedule[m][o2][3])

    return model

def getSolution(model):
    # return the solution of the model after optimization if it exists
    # return None if no solution exists
    
    # Get solution
    # ------------
    if model.status == 2:
        Completion = []
        End= []
        for _, value in enumerate(PLAN.nbOpPerMach):
            Completion.append(np.zeros(value))
        for m in range(PLAN.nbMach):
            for o in range(PLAN.nbOpPerMach[m]):
                Completion[m][o] = model.getVarByName("c_%s_%s"%(m,o)).x
        for j in range(PLAN.nbJobs):
            End.append(model.getVarByName("e_%s"%(j)).x)

        return Completion, End

    else:
        return None, [None]

# 1. Dive approach is faster than BnB

In [ ]:
def branch_and_bound_Test1(typeConst="criticMach", maxIter=2**PLAN.nbOp, rankingConstraint = []):
    # Branch and bound algorithm with extanded only one child to mimic the dive approach

    # GET THE ROOT
    ranking, end,_,_,_ = getRanking(PLAN, SD,setS=rankingConstraint)
    borneInf = PLAN.getObjective(end)
    SCHED, _ = getScheduleFromRanking(PLAN, SD, ranking)

    _, endReal = PLAN.getStartEnd(SCHED)
    borneSup = PLAN.getObjective(endReal)

    nextConst = get_constraint(ranking, SCHED, typeConst, [])
    root = TreeNode(borneSup, borneInf, None, nextConst)

    fringe = PriorityQueue()
    fringe.push(root, 0)

    bestNode = root

    iter = 0
    while len(fringe.heap) > 0 and iter < maxIter:
        priority, item = fringe.pop()

        if item.bsup < bestNode.bsup:
            bestNode = item            

        if item.nc is not None:
            if item.c is not None:
                constList = get_constraint_from_node(item)
            else:
                constList = item.nc

            for i in range(2):
                ranking, endRank,_, _, _ = getRanking(PLAN, SCHED, constraintsBB=constList+[i],setS=rankingConstraint)
                if endRank[0] == None:
                    continue

                borneInf = PLAN.getObjective(endRank)

                if borneInf < bestNode.bsup:
                    schedule, _ = getScheduleFromRanking(PLAN, SCHED, ranking)
                    _, endReal = PLAN.getStartEnd(schedule)
                    borneSup = PLAN.getObjective(endReal)
                    nextConst = get_constraint(ranking, SCHED, typeConst,constList+[i] )
                    if nextConst[0] != None:
                        n = TreeNode(borneSup, borneInf, i, nextConst, parent=item)
                        prio = - (-priority+1) # DFS
                        fringe.push(n,prio)
                        if i == 1:
                            iter += 1
                break # Only do one iteration of the loop (i.e. only one child) if the first one is feasible
        
        iter+=1

    return constList[:-3], int(borneSup), int(borneInf)

In [ ]:
# Time for the branch and bound algorithm
bnbTime = np.zeros((3,4))
for k, rc in enumerate([[0,0,0,0,0,0,0],[0,0,1,0,0,0,0],[1,1,1,1,1,1,1]]):
    for i, tc in enumerate(["CM-LV", "CM-FO", "LJ","CM-5LV"]):
        print(rc, tc)
        for j in range(2):
            start = time.time()
            constBnB, bSup, bInf = branch_and_bound_Test1(typeConst=tc, maxIter=1000, rankingConstraint = rc)
            end = time.time()
            t = end-start
            bnbTime[k,i] += t
            print(len(constBnB)/4, bSup, bInf, t)
    print("\n")

print(bnbTime)

In [ ]:
def dive_Test1(typeConst, maxIter=10, rankingConstraint=[]):
    # Dive method that impose always the same order.
    # If the order is not feasible, the constraint is removed and the constraint with the opposite order is added

    # GET THE ROOT
    ranking, end,_,_,_ = getRanking(PLAN, SD,setS=rankingConstraint)
    borneInf = PLAN.getObjective(end)
    SCHED, _ = getScheduleFromRanking(PLAN, SD, ranking)

    _, endReal = PLAN.getStartEnd(SCHED)
    borneSup = PLAN.getObjective(endReal)

    nextConst = get_constraint(ranking, SCHED, typeConst, [])
    root = TreeNode(borneSup, borneInf, None, nextConst)
    
    # Initialization of the dive
    First = True
    Infeasible = False
    
    borneInf = root.binf
    borneSup = root.bsup
    bestObj = root.bsup
    completion = None                
    constList = get_constraint_from_node(root)
    r = 0
    model = createModelInitial(SCHED, constraintsBB = constList+[r], setS=rankingConstraint)
    
    iter = 0
    while iter < maxIter:
        iter += 1
        if borneInf >= bestObj:
            message = "Break up because borne inf"
            break

        if First: # The first constraint is already added to the model
            First = False
        elif Infeasible: # The last constraint is infeasible: removed it and add the opposite constraint to the model
            model.remove(model.getConstrs()[-1])
            m = constList[-3]
            o1 = constList[-2]
            o2 = constList[-1]
            if r == 1:
                model.addConstr(model.getVarByName("c_%s_%s"%(m,o1)) >= model.getVarByName("c_%s_%s"%(m,o2)) + SCHED[m][o1][3])
                r = 0
            else:
                model.addConstr(model.getVarByName("c_%s_%s"%(m,o2)) >= model.getVarByName("c_%s_%s"%(m,o1)) + SCHED[m][o2][3])
                r = 1
        else: # Add the last constraint to the model
            r = 0 # i.e. parallel to left child instead of random.randint(0,1)
            m = constList[-3]
            o1 = constList[-2]
            o2 = constList[-1]
            if r == 0:
                model.addConstr(model.getVarByName("c_%s_%s"%(m,o1)) >= model.getVarByName("c_%s_%s"%(m,o2)) + SCHED[m][o1][3])              
            else:
                model.addConstr(model.getVarByName("c_%s_%s"%(m,o2)) >= model.getVarByName("c_%s_%s"%(m,o1)) + SCHED[m][o2][3])

        model.optimize()
        ranking, endRank = getSolution(model)
        break2 = False
        if endRank[0] == None:
            if Infeasible:
                message = "Break up because infeasible twice"
                print(message)
                break2 = True
            Infeasible = True
            continue
        else:
            Infeasible = False
        
        if break2:
            break

        borneInf = PLAN.getObjective(endRank)
        schedule, _ = getScheduleFromRanking(PLAN, SCHED, ranking)
        _, endReal = PLAN.getStartEnd(schedule)

        borneSup = PLAN.getObjective(endReal)

        nextConst = get_constraint(ranking, SCHED, typeConst,constList+[r])
        if nextConst[0] != None:
            constList  += [r] + nextConst
        else:
            message = "Break up because no more const to add"
            print(message)
            break

    return constList[:-3], int(borneSup), int(borneInf)

In [ ]:
# Time for the bdive method
diveTime = np.zeros((3,4))
for k, rc in enumerate([[0,0,0,0,0,0,0],[0,0,1,0,0,0,0],[1,1,1,1,1,1,1]]):
    for i, tc in enumerate(["CM-FO", "CM-LV", "LJ","CM-5LV"]):
        print(rc, tc)
        for j in range(2):
            start = time.time()
            constDive, bSup, bInf = dive_Test1(typeConst=tc, maxIter=1000, rankingConstraint = rc)
            end = time.time()
            t = end-start
            diveTime[k,i] += t
            print(len(constDive)/4, bSup, bInf, t)
    print("\n")
print(diveTime)

# 2. Which constraint strategy to choose?

## 2.1 Classic constraint strategies


In [ ]:
def dive_Test2_constraintStrategy(nbDive, typeConst, rankingConstraint=[], independent = False):
    # GET THE ROOT
    ranking, end,_,_,_ = getRanking(PLAN, SD,setS=rankingConstraint)
    borneInf0 = PLAN.getObjective(end)
    SCHED, _ = getScheduleFromRanking(PLAN, SD, ranking)

    _, endReal = PLAN.getStartEnd(SCHED)
    borneSup0 = PLAN.getObjective(endReal)

    nextConst = get_constraint(ranking, SCHED, typeConst, [])

    # GET STARTING NODE  

    # Save best schedule
    bestObj = borneSup0   
    bestCompletion = None
    bestEnd = None
    bestBInf = borneInf0
    bestSchedule = None
    bestConst = None

    # To plot the evolution of the dive
    binfList = [[] for i in range(nbDive)]
    bsupList = [[] for i in range(nbDive)]
    bestObjDiveList = []
    nbConstList = []

    for dive in range(nbDive):       
        # Initialization of the dive
        startTimeDiveCurr = time.time()
        First = True
        Infeasible = False
        
        # Initialization of the first node
        bestObjDive = borneSup0
        borneSup = borneSup0 # Same for each dive for this test to evaluate dives in the same conditions
        borneInf = borneInf0 # "" 
        binfList[dive].append(borneInf)        
        constList = nextConst
        r = random.randint(0,1)
        model = createModelInitial(SCHED, constraintsBB = constList+[r], setS=rankingConstraint)       
        
        while True:
            if borneSup < bestObj:
                bestObj = borneSup
                bestConst = constList[:-3]
                bestBInf = borneInf
                bestCompletion = completion
                bestEnd = endReal
                bestSchedule = schedule

            if independent:
                prune = bestObjDive
            else:
                prune = bestObj

            if borneInf >= prune:
                message = "Break up because borne inf"
                break

            if First: # The first constraint is already added to the model
                First = False
            elif Infeasible: # The last constraint is infeasible: removed it and add the opposite constraint to the model
                model.remove(model.getConstrs()[-1])
                m = constList[-3]
                o1 = constList[-2]
                o2 = constList[-1]
                if r == 1:
                    model.addConstr(model.getVarByName("c_%s_%s"%(m,o1)) >= model.getVarByName("c_%s_%s"%(m,o2)) + SCHED[m][o1][3])
                    r = 0
                else:
                    model.addConstr(model.getVarByName("c_%s_%s"%(m,o2)) >= model.getVarByName("c_%s_%s"%(m,o1)) + SCHED[m][o2][3])
                    r = 1
            else: # Add the last constraint to the model
                r = random.randint(0,1)
                m = constList[-3]
                o1 = constList[-2]
                o2 = constList[-1]
                if r == 0:
                    model.addConstr(model.getVarByName("c_%s_%s"%(m,o1)) >= model.getVarByName("c_%s_%s"%(m,o2)) + SCHED[m][o1][3])              
                else:
                    model.addConstr(model.getVarByName("c_%s_%s"%(m,o2)) >= model.getVarByName("c_%s_%s"%(m,o1)) + SCHED[m][o2][3])

            model.optimize()
            ranking, endRank = getSolution(model)
            if endRank[0] == None:
                Infeasible = True
                continue
            else:
                Infeasible = False

            borneInf = PLAN.getObjective(endRank)
            binfList[dive].append(borneInf)
            schedule, _ = getScheduleFromRanking(PLAN, SCHED, ranking)
            completion, endReal = PLAN.getStartEnd(schedule)

            borneSup = PLAN.getObjective(endReal)
            bsupList[dive].append(borneSup)
            if borneSup < bestObjDive:
                bestObjDive = borneSup

            nextConst = get_constraint(ranking, SCHED, typeConst,constList+[r])
            if nextConst[0] != None:
                constList  += [r] + nextConst
            else:
                message = "Break up because no more const to add"
                print(message)
                break
        
        bestObjDiveList.append(bestObjDive)
        nbConstList.append(len(constList)/4)

    best = [bestObj,bestCompletion,bestEnd,bestBInf,bestSchedule,bestConst]          
    
    return best, binfList, bsupList, bestObjDiveList, nbConstList

In [ ]:
def plot_test2_borne(borne, typeConst, rankingConstraint, type):
    # Plot and save the evolution of the lower/upper bound during the dive
    plt.figure()
    for i in range(len(borne)):
        plt.plot(borne[i])
    
    if typeConst == "CM-LV-M":
        plt.xlabel("Number of explored nodes")
    else:
        plt.xlabel("Number of constraints added")
        
    if type == "inf":
        plt.ylabel("Lower bound")
        y = "binf"
    elif type == "sup":
        plt.ylabel("Upper bound")
        y = "bsup"
    else:
        print("type not implemented")
        return
    #plt.title("Evolution of the lower bound during the dives \\ strategy : %s \\ set S : %s"%(typeConst, rankingConstraint))
    plt.savefig("DIVEresults/test2_" + y + "_%s_%s.png"%(typeConst, rankingConstraint[2]))
    plt.close()

In [ ]:
# Evolution of the lower and upper bound in function of the number of constraints added
# for different strategies and different sets S over 10 dives
nbDive = 10
for k, rc in enumerate([[0,0,1,0,0,0,0], [0,0,0,0,0,0,0]]): 
    for i, tc in enumerate(["CM-5LV", "CM-FO", "CM-LV", "LJ"]):
        print(rc, tc)
        start = time.time()
        bestL, binfL, bsupL, _, _ = dive_Test2_constraintStrategy(nbDive, typeConst=tc, rankingConstraint = rc, independent = True)
        end = time.time()
        t = end-start
        print(str(rc) + " " + tc + " " + str(t))

        # Save the results of the lower bound
        f = open("DIVEresults/binfList_" + tc + "_" + str(rc[2]) + ".txt","w")
        for i in range(len(binfL)):
            for j in range(len(binfL[i])):
                f.write(str(binfL[i][j])+" ")
            f.write("\n")
        f.close()

        # Save the results of the upper bound
        f = open("DIVEresults/bsupList_" + tc + "_" + str(rc[2]) + ".txt","w")
        for i in range(len(bsupL)):
            for j in range(len(bsupL[i])):
                f.write(str(bsupL[i][j])+" ")
            f.write("\n")
        f.close()

        # Plot the evolution of the lower bound and the upper bound
        plot_test2_borne(binfL, tc, rc, type="inf")
        plot_test2_borne(bsupL, tc, rc, type="sup")
        

## 2.2 CM-2LV constraint strategy

In [ ]:
def dive_Test2_cm2lv(nbDive, rankingConstraint=[], independent = False):
    typeConst = "CM-LV-M"
    # GET THE ROOT
    ranking, end,_,_,_ = getRanking(PLAN, SD,setS=rankingConstraint)
    borneInf0 = PLAN.getObjective(end)
    SCHED, _ = getScheduleFromRanking(PLAN, SD, ranking)

    _, endReal = PLAN.getStartEnd(SCHED)
    borneSup0 = PLAN.getObjective(endReal)

    nextConst = get_constraint(ranking, SCHED, typeConst, [])

    # Save best schedule
    bestObj = borneSup0   
    bestCompletion = None
    bestEnd = None
    bestSchedule = None
    bestConst = None

    # To plot the evolution of the dive
    binfList = [[] for i in range(nbDive)]
    bsupList = [[] for i in range(nbDive)]
    bestObjDiveList = []
    nbConstList = []

    for dive in range(nbDive):       
        # Initialization of the dive
        First = True
        Infeasible = 0
        
        # Initialization of the first node
        bestObjDive = borneSup0
        borneSup = borneSup0 # Same for each dive for this test to evaluate dives in the same conditions
        borneInf = borneInf0 # ""

        r = [random.choice([0, 1]) for _ in range(2)]
        constList = []
        for i in range(2):
            constList = constList + [nextConst[3*i+0], nextConst[3*i+1], nextConst[3*i+2], r[i]]
        model = createModelInitial(SCHED, constraintsBB = constList, setS=rankingConstraint)    
  
        while True:

            if borneSup < bestObj:
                bestObj = borneSup
                bestConst = constList[:-3]
                bestBInf = borneInf
                bestCompletion = completion
                bestEnd = endReal
                bestSchedule = schedule

            if independent:
                prune = bestObjDive
            else:
                prune = bestObj
            if borneInf >= prune:
                message = "Break up because borne inf"
                break

            if First: # The first constraint is already added to the model
                First = False
            elif Infeasible == 1: # Change last const x y -> x \y
                model.remove(model.getConstrs()[-1:])
                m = constList[-4]
                o1 = constList[-3]
                o2 = constList[-2]
                r1 = constList[-1]
                if r1 == 1:
                    model.addConstr(model.getVarByName("c_%s_%s"%(m,o1)) >= model.getVarByName("c_%s_%s"%(m,o2)) + SCHED[m][o1][3])
                    constList[-1] = 0
                else:
                    model.addConstr(model.getVarByName("c_%s_%s"%(m,o2)) >= model.getVarByName("c_%s_%s"%(m,o1)) + SCHED[m][o2][3])
                    constList[-1] = 1

            elif Infeasible == 2: # Change last 2 const (put initial value to the last one) x y -> x \y -> \x y
                model.remove(model.getConstrs()[-2:])
                for i in range(int(len(nextConst)/3)):
                    m = constList[-4*(i+1)]
                    o1 = constList[-4*(i+1)+1]
                    o2 = constList[-4*(i+1)+2]
                    r1 = constList[-4*(i+1)+3]

                    if r1 == 1:
                        model.addConstr(model.getVarByName("c_%s_%s"%(m,o1)) >= model.getVarByName("c_%s_%s"%(m,o2)) + SCHED[m][o1][3])
                        constList[-4*(i+1)+3] = 0
                    else:
                        model.addConstr(model.getVarByName("c_%s_%s"%(m,o2)) >= model.getVarByName("c_%s_%s"%(m,o1)) + SCHED[m][o2][3])
                        constList[-4*(i+1)+3] = 1

            elif Infeasible == 3: # x y -> x \y -> \x y -> \x \y
                model.remove(model.getConstrs()[-1:])
                m = constList[-4]
                o1 = constList[-3]
                o2 = constList[-2]
                r1 = constList[-1]
                if r1 == 1:
                    model.addConstr(model.getVarByName("c_%s_%s"%(m,o1)) >= model.getVarByName("c_%s_%s"%(m,o2)) + SCHED[m][o1][3])
                    constList[-1] = 0
                else:
                    model.addConstr(model.getVarByName("c_%s_%s"%(m,o2)) >= model.getVarByName("c_%s_%s"%(m,o1)) + SCHED[m][o2][3])
                    constList[-1] = 1

            elif Infeasible == 4: # both cannot be applied at the same time
                model.remove(model.getConstrs()[-2:])
                constList = constList[:-4]
                m = constList[-4]
                o1 = constList[-3]
                o2 = constList[-2]
                rx = constList[-1]
                if rx == 0:
                    model.addConstr(model.getVarByName("c_%s_%s"%(m,o1)) >= model.getVarByName("c_%s_%s"%(m,o2)) + SCHED[m][o1][3])
                else:
                    model.addConstr(model.getVarByName("c_%s_%s"%(m,o2)) >= model.getVarByName("c_%s_%s"%(m,o1)) + SCHED[m][o2][3])
            
            elif Infeasible == 5: # both cannot be applied at the same time
                model.remove(model.getConstrs()[-1:])
                m = constList[-4]
                o1 = constList[-3]
                o2 = constList[-2]
                r1 = constList[-1]
                if r1 == 1:
                    model.addConstr(model.getVarByName("c_%s_%s"%(m,o1)) >= model.getVarByName("c_%s_%s"%(m,o2)) + SCHED[m][o1][3])
                    constList[-1] = 0
                else:
                    model.addConstr(model.getVarByName("c_%s_%s"%(m,o2)) >= model.getVarByName("c_%s_%s"%(m,o1)) + SCHED[m][o2][3])
                    constList[-1] = 1

            elif Infeasible >= 6:
                message = "Break up because infeasible 6 times"
                break

            else: # Add the last constraint to the model
                r = [random.choice([0, 1]) for _ in range(2)]
                for i in range(int(len(nextConst)/3)):
                    constList = constList + [nextConst[3*i+0], nextConst[3*i+1], nextConst[3*i+2], r[i]]
                for i in range(int(len(nextConst)/3)):
                    rx = r[i]
                    m = nextConst[3*i+0]
                    o1 = nextConst[3*i+1]
                    o2 = nextConst[3*i+2]
                    
                    if rx == 0:
                        model.addConstr(model.getVarByName("c_%s_%s"%(m,o1)) >= model.getVarByName("c_%s_%s"%(m,o2)) + SCHED[m][o1][3])
                    else:
                        model.addConstr(model.getVarByName("c_%s_%s"%(m,o2)) >= model.getVarByName("c_%s_%s"%(m,o1)) + SCHED[m][o2][3]) 
                

            model.optimize()
            ranking, endRank = getSolution(model)
            if endRank[0] == None:
                Infeasible += 1
                continue

            else:
                Infeasible = 0
            borneInf = PLAN.getObjective(endRank)            
                
            binfList[dive].append(borneInf)
            schedule, _ = getScheduleFromRanking(PLAN, SCHED, ranking)
            completion, endReal = PLAN.getStartEnd(schedule)

            borneSup = PLAN.getObjective(endReal)
            bsupList[dive].append(borneSup)
            if borneSup < bestObjDive:
                bestObjDive = borneSup

            nextConst = get_constraint(ranking, SCHED, typeConst,constList+[r])
            if nextConst[0] == None:
                message = "Break up because no more const to add"
                break
        
        bestObjDiveList.append(bestObjDive)
        nbConstList.append(len(constList)/4)

    best = [bestObj,bestCompletion,bestEnd,bestBInf,bestSchedule,bestConst]

    return best, binfList, bsupList, bestObjDiveList, nbConstList

In [ ]:
# Evolution of the lower and upper bound in function of the number of constraints added
# for the CM-2LV constraint strategy and different sets S over 10 dives 
nbDive = 10
for k, rc in enumerate([[0,0,1,0,0,0,0], [0,0,0,0,0,0,0]]):
    tc = "CM-LV-M"
    start = time.time()
    bestL,binfL,bsupL,_,_ = dive_Test2_cm2lv(10, rc, independent = True)
    end = time.time()
    t = end-start
    print(str(rc) + " " + str(tc) + " " + str(t))
    
    # Save the results of the lower bound
    f = open("DIVEresults/binfList_" + tc + "_" + str(rc[2]) + ".txt","w")
    for i in range(len(binfL)):
        for j in range(len(binfL[i])):
            f.write(str(binfL[i][j])+" ")
        f.write("\n")
    f.close()

    # Save the results of the upper bound
    f = open("DIVEresults/bsupList_" + tc + "_" + str(rc[2]) + ".txt","w")
    for i in range(len(bsupL)):
        for j in range(len(bsupL[i])):
            f.write(str(bsupL[i][j])+" ")
        f.write("\n")
    f.close()

    # Plot and save the results
    plot_test2_borne(binfL, tc, rc, type="inf")
    plot_test2_borne(bsupL, tc, rc, type="sup")

## 2.3 Histograms

In [ ]:
def plotHistogramBS(BS, tc):
    plt.figure()
    interval = 2500
    rounded_values = [round(value / interval) * interval for value in BS]

    # Define the value ranges and bins
    min = round(np.min(BS), -3)
    maxx = round(np.max(BS), -3) + interval
    bins = np.arange(min, maxx, interval)

    # Create the histogram
    plt.hist(rounded_values, bins=bins,edgecolor='black')

    # Set the axis labels and title
    plt.xlabel("Objective value")
    plt.ylabel("Number of dives")

    plt.savefig("DIVEresults/histogramBS_%s.png"%(tc))
    plt.close()

In [ ]:
# Plot the histogram of the value of the objective for 100 dives for each constraint strategy
# Warning: change the interval of the bins in function of the instance
nbDives = 100
for i, tc in enumerate(["CM-LV", "CM-5LV", "LJ", "CM-FO", "CM-LV-M"]):
    rc = [0,0,1,0,0,0,0]
    start = time.time()
    if tc == "CM-LV-M":
        _, _, bsupL, bs, nbC = dive_Test2_cm2lv(nbDives, rc, independent = True)
    else:
        _, _, bsupL, bs, nbC = dive_Test2_constraintStrategy(nbDives, typeConst=tc, rankingConstraint = rc, independent = True)
    end = time.time()
    t = end-start
    tmin = int(t/60)
    print(str(rc) + " " + tc + " " + str(t), str(tmin))

    plotHistogramBS(bs, tc)

# 3. Run dive method for 2 hours

In [ ]:
def dive_Test3(nbDive, typeConst, rankingConstraint,timeRun):
    # GET THE ROOT
    ranking, end,_,_,_ = getRanking(PLAN, SD,setS=rankingConstraint)
    borneInf0 = PLAN.getObjective(end)
    SCHED, _ = getScheduleFromRanking(PLAN, SD, ranking)

    _, endReal = PLAN.getStartEnd(SCHED)
    borneSup0 = PLAN.getObjective(endReal)

    nextConst = get_constraint(ranking, SCHED, typeConst, [])

    # Save best schedule
    bestObj = borneSup0  
    bestBInf = borneInf0 
    bestCompletion = None
    bestEnd = None
    bestSchedule = None
    bestConst = None

    # To plot the evolution of the dive
    startTime = time.time()
    endTime = time.time()
    dive = 0
    
    while endTime-startTime < timeRun:       
        # Initialization of the dive
        First = True
        Infeasible = False
        nbConstDive = 0   
        nbConstRemoved = 0
        
        # Initialization of the first node
        bestObjDive = borneSup0
        borneSup = bestObj # The upper bound is the best objective found so far
        borneInf = borneInf0 #  Same for each dive for this test to evaluate dives in the same conditions
        constList = nextConst
        r = random.randint(0,1)
        model = createModelInitial(SCHED, constraintsBB = constList+[r], setS=rankingConstraint)       
        
        while True:
            if borneSup < bestObj:
                bestObj = borneSup
                bestConst = constList[:-3]
                bestBInf = borneInf
                bestCompletion = completion
                bestEnd = endReal
                bestSchedule = schedule
            if borneInf >= bestObj:
                message = "Break up because borne inf"
                break

            if First: # The first constraint is already added to the model
                First = False
            elif Infeasible: # The last constraint is infeasible: removed it and add the opposite constraint to the model
                model.remove(model.getConstrs()[-1])
                nbConstRemoved += 1
                m = constList[-3]
                o1 = constList[-2]
                o2 = constList[-1]
                if r == 1:
                    model.addConstr(model.getVarByName("c_%s_%s"%(m,o1)) >= model.getVarByName("c_%s_%s"%(m,o2)) + SCHED[m][o1][3])
                    r = 0
                else:
                    model.addConstr(model.getVarByName("c_%s_%s"%(m,o2)) >= model.getVarByName("c_%s_%s"%(m,o1)) + SCHED[m][o2][3])
                    r = 1
            else: # Add the last constraint to the model
                r = random.randint(0,1)
                m = constList[-3]
                o1 = constList[-2]
                o2 = constList[-1]
                if r == 0:
                    model.addConstr(model.getVarByName("c_%s_%s"%(m,o1)) >= model.getVarByName("c_%s_%s"%(m,o2)) + SCHED[m][o1][3])              
                else:
                    model.addConstr(model.getVarByName("c_%s_%s"%(m,o2)) >= model.getVarByName("c_%s_%s"%(m,o1)) + SCHED[m][o2][3])
            nbConstDive += 1

            model.optimize()
            ranking, endRank = getSolution(model)
            if endRank[0] == None:
                Infeasible = True
                continue
            else:
                Infeasible = False

            borneInf = PLAN.getObjective(endRank)
            schedule, _ = getScheduleFromRanking(PLAN, SCHED, ranking)
            completion, endReal = PLAN.getStartEnd(schedule)

            borneSup = PLAN.getObjective(endReal)
            if borneSup < bestObjDive:
                bestObjDive = borneSup
            nextConst = get_constraint(ranking, SCHED, typeConst,constList+[r])
            if nextConst[0] != None:
                constList  += [r] + nextConst
            else:
                message = "Break up because no more const to add"
                print(message)
                break
            
        endTime = time.time()
        dive += 1

    best = [bestObj,bestCompletion,bestEnd,bestBInf,bestSchedule,bestConst]
            
    return best, dive

In [ ]:
def dive_Test3_cm2lv(rankingConstraint=[], timeRun = 2*60*60):
    typeConst = "CM-LV-M"
    # GET THE ROOT
    ranking, end,_,_,_ = getRanking(PLAN, SD,setS=rankingConstraint)
    borneInf0 = PLAN.getObjective(end)
    SCHED, _ = getScheduleFromRanking(PLAN, SD, ranking)

    _, endReal = PLAN.getStartEnd(SCHED)
    borneSup0 = PLAN.getObjective(endReal)

    nextConst = get_constraint(ranking, SCHED, typeConst, [])

    # Save best schedule
    bestObj = borneSup0   
    bestCompletion = None
    bestEnd = None
    bestSchedule = None
    bestConst = None

    startTime = time.time()
    endTime = time.time()
    dive = 0
    
    while endTime-startTime < timeRun:       
        # Initialization of the dive
        First = True
        Infeasible = 0
        
        # Initialization of the first node
        borneSup = borneSup0 # Same for each dive for this test to evaluate dives in the same conditions
        borneInf = borneInf0 # "" 


        r = [random.choice([0, 1]) for _ in range(2)]
        constList = []
        for i in range(2):
            constList = constList + [nextConst[3*i+0], nextConst[3*i+1], nextConst[3*i+2], r[i]]
        model = createModelInitial(SCHED, constraintsBB = constList, setS=rankingConstraint)    
  
        
        while True:
           
            if borneSup < bestObj:
                bestObj = borneSup
                bestConst = constList[:-3]
                bestBInf = borneInf
                bestCompletion = completion
                bestEnd = endReal
                bestSchedule = schedule

            if borneInf >= bestObj:
                message = "Break up because borne inf"
                break

            if First: # The first constraint is already added to the model
                First = False
            elif Infeasible == 1: # Change last const x y -> x \y
                model.remove(model.getConstrs()[-1:])
                m = constList[-4]
                o1 = constList[-3]
                o2 = constList[-2]
                r1 = constList[-1]
                if r1 == 1:
                    model.addConstr(model.getVarByName("c_%s_%s"%(m,o1)) >= model.getVarByName("c_%s_%s"%(m,o2)) + SCHED[m][o1][3])
                    constList[-1] = 0
                else:
                    model.addConstr(model.getVarByName("c_%s_%s"%(m,o2)) >= model.getVarByName("c_%s_%s"%(m,o1)) + SCHED[m][o2][3])
                    constList[-1] = 1
                #print("1. remove const ", m,o1,o2,r1, "  new ", constList[-8:])
            elif Infeasible == 2: # Change last 2 const (put initial value to the last one) x y -> x \y -> \x y
                model.remove(model.getConstrs()[-2:])
                for i in range(int(len(nextConst)/3)):
                    m = constList[-4*(i+1)]
                    o1 = constList[-4*(i+1)+1]
                    o2 = constList[-4*(i+1)+2]
                    r1 = constList[-4*(i+1)+3]
                    #print("2. remove const ", m,o1,o2,r1)
                    if r1 == 1:
                        model.addConstr(model.getVarByName("c_%s_%s"%(m,o1)) >= model.getVarByName("c_%s_%s"%(m,o2)) + SCHED[m][o1][3])
                        constList[-4*(i+1)+3] = 0
                    else:
                        model.addConstr(model.getVarByName("c_%s_%s"%(m,o2)) >= model.getVarByName("c_%s_%s"%(m,o1)) + SCHED[m][o2][3])
                        constList[-4*(i+1)+3] = 1
                #print("new ", constList[-8:])
            elif Infeasible == 3: # x y -> x \y -> \x y -> \x \y
                model.remove(model.getConstrs()[-1:])
                m = constList[-4]
                o1 = constList[-3]
                o2 = constList[-2]
                r1 = constList[-1]
                if r1 == 1:
                    model.addConstr(model.getVarByName("c_%s_%s"%(m,o1)) >= model.getVarByName("c_%s_%s"%(m,o2)) + SCHED[m][o1][3])
                    constList[-1] = 0
                else:
                    model.addConstr(model.getVarByName("c_%s_%s"%(m,o2)) >= model.getVarByName("c_%s_%s"%(m,o1)) + SCHED[m][o2][3])
                    constList[-1] = 1
                #print("3. remove const ", m,o1,o2,r1, " new ", constList[-8:])
            elif Infeasible == 4: # both cannot be applied at the same time
                model.remove(model.getConstrs()[-2:])
                constList = constList[:-4]
                m = constList[-4]
                o1 = constList[-3]
                o2 = constList[-2]
                rx = constList[-1]
                if rx == 0:
                    model.addConstr(model.getVarByName("c_%s_%s"%(m,o1)) >= model.getVarByName("c_%s_%s"%(m,o2)) + SCHED[m][o1][3])
                else:
                    model.addConstr(model.getVarByName("c_%s_%s"%(m,o2)) >= model.getVarByName("c_%s_%s"%(m,o1)) + SCHED[m][o2][3])
                print("infeasible 4 ", m,o1,o2,rx)
            
            elif Infeasible == 5: # both cannot be applied at the same time
                model.remove(model.getConstrs()[-1:])
                m = constList[-4]
                o1 = constList[-3]
                o2 = constList[-2]
                r1 = constList[-1]
                if r1 == 1:
                    model.addConstr(model.getVarByName("c_%s_%s"%(m,o1)) >= model.getVarByName("c_%s_%s"%(m,o2)) + SCHED[m][o1][3])
                    constList[-1] = 0
                else:
                    model.addConstr(model.getVarByName("c_%s_%s"%(m,o2)) >= model.getVarByName("c_%s_%s"%(m,o1)) + SCHED[m][o2][3])
                    constList[-1] = 1
                print("infeasible 5 ", m,o1,o2,r1)
            elif Infeasible >= 6:
                print("Infeasible > 6 ")
                message = "Break up because infeasible 6 times"
                break




            else: # Add the last constraint to the model
                r = [random.choice([0, 1]) for _ in range(2)]
                for i in range(int(len(nextConst)/3)):
                    constList = constList + [nextConst[3*i+0], nextConst[3*i+1], nextConst[3*i+2], r[i]]
                for i in range(int(len(nextConst)/3)):
                    rx = r[i]
                    m = nextConst[3*i+0]
                    o1 = nextConst[3*i+1]
                    o2 = nextConst[3*i+2]
                    
                    if rx == 0:
                        model.addConstr(model.getVarByName("c_%s_%s"%(m,o1)) >= model.getVarByName("c_%s_%s"%(m,o2)) + SCHED[m][o1][3])
                    else:
                        model.addConstr(model.getVarByName("c_%s_%s"%(m,o2)) >= model.getVarByName("c_%s_%s"%(m,o1)) + SCHED[m][o2][3]) 
                

            model.optimize()
            ranking, endRank = getSolution(model)
            if endRank[0] == None:
                Infeasible += 1
                #print("Infeasible ", nextConst)
                continue
            else:
                Infeasible = 0

            borneInf = PLAN.getObjective(endRank)            
            schedule, _ = getScheduleFromRanking(PLAN, SCHED, ranking)
            completion, endReal = PLAN.getStartEnd(schedule)

            borneSup = PLAN.getObjective(endReal)

            nextConst = get_constraint(ranking, SCHED, typeConst,constList+[r])
            if nextConst[0] == None:
                message = "Break up because no more const to add"
                break

        endTime = time.time()
        dive += 1

        #print(message, " nbConst = ", len(constList)/4, " objective = ", int(bestObjDive))

    best = [bestObj,bestCompletion,bestEnd,bestBInf,bestSchedule,bestConst]


    return best, dive

In [ ]:
# Run the dive method for different constraint strategies
divesObj, divesCompletion,divesEnd, divesInf, divesSched, divesConst, divesTC, divesNB  = [], [], [], [], [], [], [], []

for i, tc in enumerate(["CM-FO", "CM-LV", "CM-5LV", "CM-LV-M"]):
    rc = [0,0,1,0,0,0,0]
    timeRun = 60*60*2
    if tc == "CM-LV-M":
        best, dive = dive_Test3_cm2lv(rankingConstraint = rc, timeRun = timeRun)
    else:
        best, dive = dive_Test3(1, typeConst=tc, timeRun = timeRun, rankingConstraint = rc)
    divesNB.append(dive)
    divesSched.append(best[4])
    divesObj.append(best[0])
    divesEnd.append(best[2])
    divesCompletion.append(best[1])
    divesConst.append(best[5])
    divesTC.append(tc)
    divesInf.append(best[3])
    print(tc, int(best[0]), int(best[3]), len(best[5])/4, dive)

# Find the best constraint strategy
bestIdx = np.argmin(divesObj)
BESTOBJ = divesObj[bestIdx]
BESTSCHED = divesSched[bestIdx]
BESTEND = divesEnd[bestIdx]
BESTCOMPLETION = divesCompletion[bestIdx]
BESTCONST = divesConst[bestIdx]
BESTTC = divesTC[bestIdx]

# Save the results
f = open("DIVEresults/dives_2h.txt","w")
f.write(str(BESTSCHED))
f.write("\n")
f.write(str(BESTOBJ))
f.write("\n")
f.write(str(BESTEND))
f.write("\n")
f.write(str(BESTCOMPLETION))
f.write("\n")
f.write(str(BESTCONST))
f.write("\n")
f.write(str(BESTTC))
f.write("\n")
f.write(str(BESTRC))
f.write("\n")
f.close()

# Save the best schedule in another file
f = open("DIVEresults/dives_2h_sched.txt","w")
f.write(str(BESTSCHED))
f.close()

# 4. Big BnB

## 4.1 To increase lower bound

In [ ]:
# Get the best schedule obtained with the dive approach
currPath = os.getcwd()
with open(currPath + "/DIVEresults/dives_2h_sched.txt", 'r') as file:
    contents = file.read()
BESTSCHED = ast.literal_eval(contents) # Convert the contents into a list of lists of tuples

# Run the branch and bound method
for i, es in enumerate(["CM-FO", "CM-LV"]):
    bn, _, _,prune,_,leaf, _, _,_ = branch_and_bound(typeConst=es,firstSchedule=BESTSCHED, maxIter=0, prioType="BFS", rankingConstraint=[0,0,1,0,0,0,0], timeRun=60*60*16)
    print(es, bn.binf, bn.bsup, prune, leaf)


## 4.2 To decrease objective

In [ ]:
# Get the best schedule obtained with the dive approach
currPath = os.getcwd()
with open(currPath + "/DIVEresults/dives_2h_sched.txt", 'r') as file:
    contents = file.read()
BESTSCHED = ast.literal_eval(contents) # Convert the contents into a list of lists of tuples

# Run the branch and bound method
for i, es in enumerate(["CM-LV", "CM-5LV"]):
    bn, _, _,prune,_,leaf, _, _,_ = branch_and_bound(typeConst=es,firstSchedule=BESTSCHED, maxIter=0, prioType="GS-up", rankingConstraint=[0,0,1,0,0,0,0], timeRun=60*60*2)
    print(es, bn.binf, bn.bsup, prune, leaf)